In [228]:
import random
import sys
import os
import json
import re

In [229]:
filepath = '/scratch1/nsuresh/DL/'
with open(filepath + 'training_label.json', 'r') as f:
    file = json.load(f)

word_count = {}
for d in file:
    for s in d['caption']:
        word_sentence = re.sub('[.!,;?]]', ' ', s).split()
        for word in word_sentence:
            word = word.replace('.', '') if '.' in word else word
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

In [230]:
word_dict = {}
for word in word_count:
    if word_count[word] > 10:
        word_dict[word] = word_count[word]
useful_tokens = [('<PAD>', 0), ('<SOS>', 1), ('<EOS>', 2), ('<UNK>', 3)]
i2w = {i + len(useful_tokens): w for i, w in enumerate(word_dict)}
w2i = {w: i + len(useful_tokens) for i, w in enumerate(word_dict)}
for token, index in useful_tokens:
    i2w[index] = token
    w2i[token] = index

In [231]:
def annotate(sentence):
    sentence = re.sub(r'[.!,;?]', ' ', sentence).split()
    for i in range(len(sentence)):
        if sentence[i] not in word_dict:
            sentence[i] = 3
        else:
            sentence[i] = w2i[sentence[i]]
    sentence.insert(0, 1)
    sentence.append(2)
    return sentence

In [232]:
label_json = filepath + 'training_label.json'
annotated_caption = []
with open(label_json, 'r') as f:
    label = json.load(f)
for d in label:
    for s in d['caption']:
        s = annotate(s)
        annotated_caption.append((d['id'], s))

In [233]:
avi_data = {}
training_feats = filepath + '/training_data/feat'
files = os.listdir(training_feats)
for file in files:
    value = np.load(os.path.join(training_feats, file))
    avi_data[file.split('.npy')[0]] = value

In [237]:
class training_data(Dataset):
    def __init__(self):
        self.avi = avi_data
        self.data_pair = annotated_caption
    def __len__(self):
        return len(self.data_pair)
    def __getitem__(self, idx):
        assert (idx < self.__len__())
        avi_file_name, sentence = self.data_pair[idx]
        data = torch.Tensor(self.avi[avi_file_name])
        data += torch.Tensor(data.size()).random_(0, 2000)/10000.
        return torch.Tensor(data), torch.Tensor(sentence)